In [ ]:
# Install necessary packages
%pip install pandas plotly nbformat

In [ ]:
import json
import plotly.graph_objects as go
import pandas as pd

hover_label_dict =dict(
    align='left',
    bgcolor="white",
    font_size=12,
    font_family="Arial"
)

# Load both JSON files
with open('../profile_results.json', 'r') as file:
    profile_data = json.load(file)

with open('../profiler_n_frames.json', 'r') as file:
    frame_data = json.load(file)

# Create DataFrame for frame data
frames_df = pd.DataFrame(frame_data['frames'])

# Check if we have profile data
has_profile_data = len(profile_data['profiles']) > 0

if has_profile_data:
    # Create DataFrame for profile data only if we have profiles
    profiles_df = pd.DataFrame(profile_data['profiles'])
    
    # Define custom sorting order
    function_order = ['BeginScene', 'Present', 'EndScene']
    profiles_df['sort_order'] = profiles_df['name'].map({name: i for i, name in enumerate(function_order)})
    profiles_df = profiles_df.sort_values('sort_order').drop('sort_order', axis=1)
    
    # Visualization 1: Profile Overview
    fig1 = go.Figure()
    
    for metric in ['minMs', 'averageMs', 'maxMs']:
        if metric in profiles_df.columns:
            fig1.add_trace(go.Bar(
                name=metric.replace('Ms', ' Time'),
                x=profiles_df['name'] if 'name' in profiles_df.columns else [],
                y=profiles_df[metric],
                text=profiles_df[metric].round(2),
                textposition='auto',
            ))
    
    fig1.update_layout(
        title='Profile Timing Overview',
        barmode='group',
        xaxis_title='Function',
        yaxis_title='Time (ms)',
        yaxis_type='log',
        hoverlabel=hover_label_dict
    )
    fig1.show()

# Visualization 2: Frame Times Distribution
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=frames_df['frame_number'],
    y=frames_df['frame_time_ms'],
    mode='lines+markers',
    name='Frame Time',
    hovertemplate='Frame %{x}<br>Time: %{y:.2f}ms'
))

# Add horizontal lines for statistics
stats = frame_data['statistics']
for stat, color in [
    ('average_frame_time_ms', 'green'),
    ('min_frame_time_ms', 'blue'),
    ('max_frame_time_ms', 'red')
]:
    value = stats[stat]
    fig2.add_hline(
        y=value,
        line_dash="dash",
        line_color=color,
        annotation_text=f'{stat.replace("_frame_time_ms", "").title()}: {value:.2f}ms'
    )

fig2.update_layout(
    title='Frame Times Over Time',
    xaxis_title='Frame Number',
    yaxis_title='Frame Time (ms)',
    hoverlabel=hover_label_dict
)

# Visualization 3: FPS Statistics
fig3 = go.Figure()

fps_stats = {
    k: v for k, v in frame_data['statistics'].items() 
    if k.startswith('fps_')
}

fig3.add_trace(go.Bar(
    x=list(fps_stats.keys()),
    y=list(fps_stats.values()),
    text=[f'{v:.1f}' for v in fps_stats.values()],
    textposition='auto'
))

fig3.update_layout(
    title='FPS Statistics',
    xaxis_title='Metric',
    yaxis_title='FPS',
    yaxis_type='log',
    hoverlabel=hover_label_dict
)

# Visualization 4: Box Plot of Frame Times
fig4 = go.Figure()

# Calculate all quartiles including min (q0) and max (q4)
quartiles = frames_df['frame_time_ms'].quantile([0, 0.25, 0.5, 0.75, 1])

fig4.add_trace(go.Box(
    y=frames_df['frame_time_ms'],
    name='Frame Times',
    boxpoints='all',
    jitter=0.3,
    pointpos=-1.8,
    boxmean=True,
    marker=dict(color='lightblue'),
    line=dict(color='blue'),
    hovertemplate='Time: %{y:.2f}ms<extra></extra>'
))

# Add lines for all quartiles including min and max
for q, (label, color) in enumerate([
    ('Min (Q0)', 'gray'), 
    ('25th', 'purple'), 
    ('Median', 'red'), 
    ('75th', 'orange'),
    ('Max (Q4)', 'darkgray')
]):
    fig4.add_hline(
        y=quartiles.iloc[q],
        line_dash="dash",
        line_color=color,
        annotation_text=f"{label}: {quartiles.iloc[q]:.2f}ms",
        annotation_position="right"
    )

# Update layout with adjusted y-axis range for better distribution
fig4.update_layout(
    title='Frame Time Distribution',
    yaxis_title='Frame Time (ms)',
    yaxis=dict(
        range=[
            quartiles.iloc[1] - (quartiles.iloc[4] - quartiles.iloc[1]) * 0.1,  # Q1 - 10% of range from Q1 to Q4
            quartiles.iloc[4] + (quartiles.iloc[4] - quartiles.iloc[1]) * 0.1   # Q4 + 10% of range from Q1 to Q4
        ]
    ),
    showlegend=False,
    hovermode='y',
    hoverlabel=hover_label_dict
)

# Display session info and remaining figures
print(f"Session: {profile_data['session']}")
print(f"Timestamp: {profile_data['timestamp']}\n")

fig2.show()
fig3.show()
fig4.show()
